In [1]:
!pip -q install transformers datasets accelerate evaluate peft bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.2 MB/s eta 0:00:00


In [3]:
import os, random
import numpy as np
import torch

def set_seed(seed=79):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

set_seed(79)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)


cpu


## text prepossing
I will use the novel *The Leavenworth Case* by Anna Katharine Green from Prtoject Gutenberg. The link for this novel is https://www.gutenberg.org/ebooks/4047


In [5]:
import requests

url = "https://www.gutenberg.org/cache/epub/4047/pg4047.txt"
raw = requests.get(url).text
# print first 1000 char for a glance
print(raw[:1000])

﻿The Project Gutenberg eBook of The Leavenworth Case
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Leavenworth Case

Author: Anna Katharine Green

Release date: May 1, 2003 [eBook #4047]
                Most recently updated: July 31, 2022

Language: English

Credits: David Moynihan, and David Widger


*** START OF THE PROJECT GUTENBERG EBOOK THE LEAVENWORTH CASE ***




                       THE LEAVENWORTH CASE

                         A LAWYER’S STORY


                      By Anna Katharine Green




[Illustration: “I did not go, but st


In [6]:
import re

START_re = re.compile(r"\*\*\*\s*START OF (THIS|THE) PROJECT GUTENBERG EBOOK.*?\*\*\*", re.I)
END_re = re.compile(r"\*\*\*\s*END OF (THIS|THE) PROJECT GUTENBERG EBOOK.*?\*\*\*", re.I)

# first step, strip boilerplate
def strip_boil(text):
  m1 = START_re.search(text)
  m2 = END_re.search(text)
  if m1 and m2 and m1.end() < m2.start():
    return text[m1.end():m2.start()]
  return text

# secondly, remve illustrations and context
def remove_front(text):
  text = text.replace("\r\n", "\n").replace("\r", "\n")
  text = re.sub(r"^\s*\[Illustration:.*?\]\s*$", "", text, flags=re.I | re.M)
  text = re.sub(
        r"(?is)\n\s*CONTENTS\s*\n.*?(?=\n\s*BOOK\s+[IVXLC]+\b|\n\s*CHAPTER\s+[IVXLC0-9]+\b|\n\s*BOOK\s+I\b)",
        "\n",
        text
    )
  text = re.sub(
        r"(?is)\n\s*Illustrations\s*\n.*?(?=\n\s*BOOK\s+[IVXLC]+\b|\n\s*CHAPTER\s+[IVXLC0-9]+\b|\n\s*BOOK\s+I\b)",
        "\n",
        text
    )
  return text

# third, locate start of story and trim to it
def trim_start(text):
  anchors = [
        r"\n\s*BOOK\s+I\.\s+THE\s+PROBLEM\s*\n",
    ]
  for a in anchors:
      m = re.search(a, text, flags=re.I)
      if m:
          return text[m.start():]
  return text

# forth, clean \n and too many white space
def clean_space(text):
  text = re.sub(r"[ \t]+\n", "\n", text)
  text = re.sub(r"\n{3,}", "\n\n", text)
  return text.strip()

# prepossing
def preprocess(raw):
  text = strip_boil(raw)
  text = remove_front(text)
  text = trim_start(text)
  text = clean_space(text)
  return text

clean_text = preprocess(raw)
print(clean_text[:1000])



BOOK I. THE PROBLEM

I. “A GREAT CASE”

      “A deed of dreadful note.”

                                Macbeth.

I had been a junior partner in the firm of Veeley, Carr & Raymond,
attorneys and counsellors at law, for about a year, when one
morning, in the temporary absence of both Mr. Veeley and Mr. Carr,
there came into our office a young man whose whole appearance was
so indicative of haste and agitation that I involuntarily rose at
his approach and impetuously inquired:

“What is the matter? You have no bad news to tell, I hope.”

“I have come to see Mr. Veeley; is he in?”

“No,” I replied; “he was unexpectedly called away this morning to
Washington; cannot be home before to-morrow; but if you will make
your business known to me----”

“To you, sir?” he repeated, turning a very cold but steady eye on
mine; then, seeming to be satisfied with his scrutiny, continued,
“There is no reason why I shouldn’t; my business is no secret. I
came to inform him that Mr. Leavenworth is dead.”



In [8]:
os.makedirs("data", exist_ok=True)

with open("data/clean_text.txt", "w", encoding="utf-8") as f:
    f.write(clean_text)

print("saved to data/clean_text.txt")

saved to data/clean_text.txt
